In [ ]:
# # Starting again, trying to solve it all...

# import torch
# import torch.nn as nn
# import scipy.optimize as opt
# import numpy as np
# import sys
# sys.path.append("../")
# from ddn.pytorch.node import *
# import warnings
# warnings.filterwarnings('ignore')
        
# def objective(W, y):
#     """
#     f(W,y) = y^T * (D-W) * y / y^T * D * y
#     """
#     # W is an NxN symmetrical matrix with W(i,j) = w_ij
#     D = W.sum(1).diag() # D is an NxN diagonal matrix with d on diagonal, for d(i) = sum_j(w(i,j))
#     L = D - W
    
#     return torch.div(torch.mm(torch.mm(torch.t(y), L),y),torch.mm(torch.mm(torch.t(y),D),y))
    
# def equality_constraints(W, y):
#     """
#     subject to y^T * D * 1 = 0
#     """
#     # Ensure correct size and shape of y... scipy minimise flattens y         
#     N = W.size(dim=0)
    
#     #x is an NxN symmetrical matrix with W(i,j) = w_ij
#     D = W.sum(1).diag() # D is an NxN diagonal matrix with d on diagonal, for d(i) = sum_j(w(i,j))
#     ONE = torch.ones(N,1)   # Nx1 vector of all ones
    
#     return torch.mm(torch.mm(torch.t(y),D), ONE)

# def solve(W):
#     """
#     Minimise the objective, by solving the second smallest eigenvector of laplacian
#     (D-W)*y = lambda * D * y
#     becomes D^-0.5 * (D-W) * D^-0.5 * y = lambda * y
#     """
#     # normalised laplacian
#     D = W.sum(1).diag()
#     L = D - W
#     # Solve using torch.linalg.eigh?
#     eigval, eigvec = torch.linalg.eig(L)

#     val = [np.round(i.real,4) for i in eigval]

#     vec = []
#     for i in range(len(eigvec)):
#         v = [np.round(n.real,4) for n in eigvec[:,i]]
#         vec.append(v)

#     s = list(val).copy()
#     s.sort()

#     for i in range(len(eigval)):
#         if val[i] == s[1]:
#             second_smallest_eigval_vec = vec[i]
#             break
    
#     return torch.tensor(second_smallest_eigval_vec)


In [1]:
"""
Sample code automatically generated on 2022-03-07 03:07:10

by www.matrixcalculus.org

from input

d/dy D.^-0.5*(D-W)*D.^-0.5*y = D.^(-0.5)*(D-W)*D.^(-0.5)

where

D is a matrix
W is a matrix
y is a vector
"""

from __future__ import division, print_function, absolute_import

import torch
import autograd.numpy as np

def fAndG(D, W, y):
    assert isinstance(D, np.ndarray)
    dim = D.shape
    assert len(dim) == 2
    D_rows = dim[0]
    D_cols = dim[1]
    assert isinstance(W, np.ndarray)
    dim = W.shape
    assert len(dim) == 2
    W_rows = dim[0]
    W_cols = dim[1]
    assert isinstance(y, np.ndarray)
    dim = y.shape
    assert len(dim) == 1
    y_rows = dim[0]
    assert D_cols == W_cols == D_rows == W_rows
    assert D_cols == y_rows == W_cols == D_rows == W_rows

    T_0 = (D ** -0.5)
    T_1 = (D - W)
    functionValue = (T_0) @ ((T_1) @ ((T_0) @ (y)))
    gradient = ((T_0) @ (T_1)) @ (T_0)

    return functionValue, gradient

def checkGradient(D, W, y):
    # numerical gradient checking
    # f(x + t * delta) - f(x - t * delta) / (2t)
    # should be roughly equal to inner product <g, delta>
    t = 1E-6
    delta = np.random.randn(3)
    f1, _ = fAndG(D, W, y + t * delta)
    f2, _ = fAndG(D, W, y - t * delta)
    f, g = fAndG(D, W, y)
    print('approximation error',
          np.linalg.norm((f1 - f2) / (2*t) - np.tensordot(g, delta, axes=1)))
    

def solve(W):
    """
    Minimise the objective, by solving the second smallest eigenvector of laplacian
    (D-W)*y = lambda * D * y
    becomes D^-0.5 * (D-W) * D^-0.5 * y = lambda * y
    """
    # laplacian
    D = W.sum(1).diag()
    L = D - W
    # Solve using torch.linalg.eigh?
    eigval, eigvec = np.linalg.eig(L)

    val = [np.round(i.real,4) for i in eigval]

    vec = []
    for i in range(len(eigvec)):
        v = [np.round(n.real,4) for n in eigvec[:,i]]
        vec.append(v)

    s = list(val).copy()
    s.sort()

    for i in range(len(eigval)):
        if val[i] == s[1]:
            second_smallest_eigval_vec = vec[i]
            break

    return second_smallest_eigval_vec

def generateRandomData():
    # D = np.random.rand(3, 3)
    W = 5 * torch.tensor(np.random.rand(3, 3))
    D = W.sum(1).diag()
    y = solve(W)

    return D.numpy(), W.numpy(), np.asarray(y)

In [2]:
D, W, y = generateRandomData()

print(D)
print(W)
print(y)


value, grad = fAndG(D,W,y)

functionValue, gradient = fAndG(D, W, y)
print('functionValue = ', functionValue)
print('gradient = ', gradient)

print('numerical gradient checking ...')
checkGradient(D, W, y)

[[ 6.18484682  0.          0.        ]
 [ 0.          7.6879524   0.        ]
 [ 0.          0.         10.20016319]]
[[0.99548008 3.07876329 2.11060344]
 [3.99313814 0.99689913 2.69791514]
 [4.61491334 2.1901351  3.39511475]]
[-0.3538 -0.0662  0.7005]
functionValue =  [nan nan nan]
gradient =  [[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
numerical gradient checking ...
approximation error nan


/data/gwales/anaconda3/envs/ddn/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: divide by zero encountered in power
/data/gwales/anaconda3/envs/ddn/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in matmul
/data/gwales/anaconda3/envs/ddn/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in matmul


In [3]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.
